In [1]:
import numpy as np
import matplotlib.pyplot as plt
import  tkinter as tk

# Parameters
m = 1
v = 10
N_part = 625
m_disk = 10
R_disk = 10
L = 260
sigma = 1
epsilon = 1
T_tot = 400
dt = 0.005
trajectory = []
msd_list = []


#  visualization setup
#window_size = 600
window = tk.Tk()
canvas = tk.Canvas(window, width=L, height=L, bg="white")
canvas.pack()


# Initialize disk
disk = {'x': L / 2, 'y': L / 2, 'vx': 0, 'vy': 0}
disk_obj = canvas.create_oval(disk['x'] - R_disk, disk['y'] - R_disk,
                              disk['x'] + R_disk, disk['y'] + R_disk, fill="black")

# Initialize particles on a grid
particles = []
particle_objs = []
for i in range(int(np.sqrt(N_part))):
    for j in range(int(np.sqrt(N_part))):
        x = i * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        y = j * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        angle = np.random.uniform(0, 2 * np.pi)
        vx = v * np.cos(angle)
        vy = v * np.sin(angle)
        particle = {'x': x, 'y': y, 'vx': vx, 'vy': vy}
        particles.append(particle)
        particle_obj = canvas.create_oval(x - 2, y - 2, x + 2, y + 2, fill="blue")
        particle_objs.append(particle_obj)

# Remove particles within the disk radius plus 3*sigma
particles = [p for p in particles if np.sqrt((p['x'] - disk['x']) ** 2 + (p['y'] - disk['y']) ** 2) > (R_disk + 3 * sigma)]




# Plot trajectory and MSD
def plot_results():
    # Plot disk's trajectory
    trajectory_x, trajectory_y = zip(*trajectory)
    plt.figure(figsize=(6, 5))  # 创建单独的图窗口
    plt.plot(trajectory_x, trajectory_y, color='blue')
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.title("Disk Trajectory in Cartesian Plane")
    plt.grid(True)
    plt.savefig( 'disk_trajectory.png') 
    plt.show()
    plt.pause(1)
    plt.close() 

    # MSD Plot
    plt.figure(figsize=(6, 5)) 
    time = np.arange(1, len(msd_list) + 1) * dt
    plt.plot(time, msd_list, color='red')
    plt.xlabel("Time (t)")
    plt.ylabel("Mean Square Displacement (MSD)")
    plt.title("Mean Square Displacement over Time")
    plt.grid(True)
    plt.savefig('msd_over_time.png')
    plt.show()
    plt.pause(1)
    plt.close()

# Lennard-Jones force calculation
def lennard_jones_force(disk, particle):
    dx = disk['x'] - particle['x']
    dy = disk['y'] - particle['y']
    r = np.sqrt(dx ** 2 + dy ** 2) - R_disk
    if r < 3 * sigma:
        F = 24 * epsilon * ((2 * (sigma / r) ** 12) - (sigma / r) ** 6) / r
        fx = F * dx / r
        fy = F * dy / r
        return fx, fy
    return 0, 0


trajectory = []
msd_list = []
time_steps = int(T_tot / dt)


# Main loop with Tkinter refresh
step = 0
running = True

while running and step < time_steps:
    # Record disk's position for trajectory and MSD calculation
    trajectory.append((disk['x'], disk['y']))

    # Calculate MSD at each step
    if step > 0:
        initial_x, initial_y = trajectory[0]
        msd = (disk['x'] - initial_x) ** 2 + (disk['y'] - initial_y) ** 2
        msd_list.append(msd)

    # Update disk position
    disk_fx, disk_fy = 0, 0
    for i, particle in enumerate(particles):
        fx, fy = lennard_jones_force(disk, particle)
        disk_fx += fx
        disk_fy += fy
        particle['vx'] += -fx / m * dt
        particle['vy'] += -fy / m * dt
        particle['x'] += particle['vx'] * dt
        particle['y'] += particle['vy'] * dt

        # Boundary conditions for particles (reflecting)
        if particle['x'] <= 0 or particle['x'] >= L:
            particle['vx'] *= -1
        if particle['y'] <= 0 or particle['y'] >= L:
            particle['vy'] *= -1

        # Update particle position in Tkinter
        canvas.coords(particle_objs[i], particle['x'] - 2, particle['y'] - 2,
                      particle['x'] + 2, particle['y'] + 2)

    # Update disk velocity and position based on net force
    disk['vx'] += disk_fx / m_disk * dt
    disk['vy'] += disk_fy / m_disk * dt
    disk['x'] += disk['vx'] * dt
    disk['y'] += disk['vy'] * dt

    # Boundary conditions for the disk (reflecting)
    if disk['x'] <= R_disk or disk['x'] >= L - R_disk:
        disk['vx'] *= -1
    if disk['y'] <= R_disk or disk['y'] >= L - R_disk:
        disk['vy'] *= -1

    # Update disk position in Tkinter
    canvas.coords(disk_obj, disk['x'] - R_disk, disk['y'] - R_disk, disk['x'] + R_disk, disk['y'] + R_disk)
    canvas.create_oval(disk['x'] - 0.2, disk['y'] - 0.2, disk['x'] + 0.2, disk['y'] + 0.2, fill="red", outline="red")

    if step % 3000 == 0:
        print(f'Time step: {step}, Disk position: ({disk["x"]}, {disk["y"]})')

    # Update Tkinter display
    window.update_idletasks()
    window.update()

    # Increment step
    step += 1

# End of simulation
plot_results()
window.mainloop()




Time step: 0, Disk position: (130.0, 130.0)
Time step: 3000, Disk position: (139.812287340959, 141.77377871007124)
Time step: 6000, Disk position: (157.73424229314568, 156.32696896959635)
Time step: 9000, Disk position: (144.29104982680326, 163.3878535790225)
Time step: 12000, Disk position: (153.62227270056533, 153.7934952275477)
Time step: 15000, Disk position: (158.40786844289138, 148.30473653446776)


2024-11-13 08:59:43.175 Python[2452:59598] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-13 08:59:43.175 Python[2452:59598] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Time step: 18000, Disk position: (154.5147458772024, 160.80192121459174)
Time step: 21000, Disk position: (119.70957297595652, 154.61413424229477)
Time step: 24000, Disk position: (142.43905331197516, 137.274944803258)
Time step: 27000, Disk position: (146.85994825601432, 116.46131947057306)
Time step: 30000, Disk position: (158.27077328962508, 123.13830764943621)
Time step: 33000, Disk position: (179.66380119054494, 133.38134542833632)
Time step: 36000, Disk position: (166.42202446064132, 165.23117976581784)
Time step: 39000, Disk position: (173.09405357014776, 163.0101994496476)
Time step: 42000, Disk position: (195.00891751066797, 184.89519767452302)
Time step: 45000, Disk position: (190.09044220795937, 200.9868883106773)
Time step: 48000, Disk position: (174.68536025837074, 198.88410392716702)
Time step: 51000, Disk position: (204.44200132082906, 216.8929423266986)
Time step: 54000, Disk position: (205.13848533494922, 206.99401796149505)
Time step: 57000, Disk position: (201.610054

: 